<a href="https://colab.research.google.com/github/stepanjaburek/workingpaper_czech_psp_speeches/blob/main/Translate_Opus_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece sacremoses torch tqdm

import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm.notebook import tqdm
import torch

In [ ]:
def translate_csv(file_path, source_lang='cs', target_lang='en', batch_size=8):
    # Load data and model
    df = pd.read_csv(file_path)
    model_name = f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

    def translate_batch(texts):
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            return tokenizer.batch_decode(model.generate(**inputs), skip_special_tokens=True)

    # Translate in batches
    translations = []
    for i in tqdm(range(0, len(df), batch_size)):
        translations.extend(translate_batch(df['context_full'][i:i + batch_size].tolist()))

    df['translated_text'] = translations
    return df


In [ ]:
translated_df = translate_csv('/content/green.csv')
translated_df.to_csv('translated_green.csv', index=False)